In [1]:
!pip install fastf1
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 0.7.3 requires websockets>=14.0, but you have websockets 13.1 which is incompatible.


In [2]:
import fastf1
from fastf1 import get_event
from collections import defaultdict
import numpy as np

# Choose the year and event
event = get_event(2024, 'Emilia Romagna Grand Prix')
session_quali = event.get_session('Q')
session_race = event.get_session('R')

# Load data
session_quali.load()
session_race.load()

# Get qualifying and race results
quali_results = session_quali.results
race_results = session_race.results

# Store driver-wise position changes
driver_changes = defaultdict(list)

# Build mapping: driver number -> abbreviation
driver_abbr = {
    row['DriverNumber']: row['Abbreviation'] for _, row in race_results.iterrows()
}

# Populate driver_changes
for _, row in quali_results.iterrows():
    number = row['DriverNumber']
    quali_pos = row['Position']

    # Check if driver has race result
    race_row = race_results[race_results['DriverNumber'] == number]
    if not race_row.empty:
        finish_pos = int(race_row.iloc[0]['Position'])
        change = int(quali_pos) - finish_pos  # Positive = lost positions
        driver_changes[driver_abbr[number]].append(change)

# Calculate averages and format as dictionary
average_position_change_emr = {
    driver: round(np.mean(changes), 1) for driver, changes in driver_changes.items()
}

# Print result in the required format
print("average_position_change = {")
for driver, avg_change in average_position_change_emr.items():
    print(f'    "{driver}": {avg_change},')
print("}")


req         WARNING 	DEFAULT CACHE ENABLED! (24.0 KB) /root/.cache/fastf1
core           INFO 	Loading data for Emilia Romagna Grand Prix - Qualifying [v3.5.3]
INFO:fastf1.fastf1.core:Loading data for Emilia Romagna Grand Prix - Qualifying [v3.5.3]
req            INFO 	No cached data found for session_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
INFO:fastf1.api:Fetching session info data...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
INFO:fastf1.api:Fetching driver list...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
DEBUG:fastf1.ergast:Failed to parse timestamp '

average_position_change = {
    "VER": 0.0,
    "PIA": -2.0,
    "NOR": 1.0,
    "LEC": 1.0,
    "SAI": 0.0,
    "RUS": -1.0,
    "TSU": -3.0,
    "HAM": 2.0,
    "RIC": -4.0,
    "HUL": -1.0,
    "PER": 3.0,
    "OCO": -2.0,
    "STR": 4.0,
    "ALB": -6.0,
    "GAS": -1.0,
    "BOT": -2.0,
    "ZHO": 2.0,
    "MAG": 6.0,
    "ALO": 0.0,
    "SAR": 3.0,
}
